# Pre-requisite

The following notebook assumes you have [configured remotes for rclone](../aws/rclone/).

We will assume the following variables. It will be loaded via `source .env` or other means:

```bash
# .env

# rclone remotes
REMOTE_AWS="aws"           # configured with STANDARD or INTELLIGENT_TIERING storage cls
REMOTE_AWS_DEEP="aws_deep" # configured with DEEP_GLACIER as storage class
REMOTE_GDRIVE="gdrive"

# root folders & test objects
ROOT_AWS="test-bucket"     # Best practice: should only be bucket name, no nested folder
ROOT_LOCAL="test_objects"
ROOT_GDRIVE="ronaldstouffer"

TEST_FOLDER="Becki_Port_april2021"

# configure some flags for rclone
FLAGGED_RCLONE="rclone -vv"  # we can also put --dry-run for no actual changes

# before any codeblock:
source .env
FLAGGED_RCLONE=${FLAGGED_RCLONE:-"rclone"}
```

# Copying from one place to another

`rclone copy [remote_src:]src_dir [remote_dest:]dest_dir`
- If no remote is specified, then it means we're performing on local filesystem.
- `rclone copy` works as if you put a trailing `/`
- rclone avoids transfering files that are identical from src and dest
  - Size & (modification time OR MD5 SUM)

In [ ]:
# trailing / demonstration
mkdir abc
touch abc/d.txt abc/hello-world.txt
# abc/d.txt  abc/hello-world.txt
$FLAGGED_RCLONE abc def

The copy results will be: `def/d.txt  def/hello-world.txt`, equivalent to:

In [ ]:
$FLAGGED_RCLONE abc/* def/
cp -r abc/ def/
cp abc/* def

# Deleting files

2 commands:
- `rclone delete [remote:]path/to/dir`: deletes the files in given path. Doesn't delete directory structure
- `rclone purge [remote:]path/to/dir`: acts like `rm -rf path/to/dir`.
  - Cannot use selections (because it's deleting the whole directory)

